## Import libraries

In [1]:
import sys
import threading
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from simworld.config import Config
from simworld.communicator.communicator import Communicator
from simworld.communicator.unrealcv import UnrealCV
from simworld.traffic.controller.traffic_controller import TrafficController
from simworld.traffic.base.traffic_signal import TrafficSignalState
from simworld.llm.a2a_llm import A2ALLM
from simworld.map.map import Map
from simworld.agent.humanoid import Humanoid
from simworld.utils.vector import Vector
from simworld.local_planner.local_planner import LocalPlanner

## Init

In [2]:
config = Config()
communicator = Communicator(UnrealCV())

INFO:__init__:230:Got connection confirm: b'connected to gym_citynav'


=>Info: using ip-port socket


## World Generation

In [4]:
communicator.generate_world('/data4/dylanlu/developer/SimWorld/simworld/data/progen_world.json', config['citygen.ue_asset_path'], run_time=True)

2025-12-05 18:26:10,933 - SimWorld.JsonLoader - WARNING - File not found at 'ue_assets.json', falling back to default location


{'GEN_BP_Box2_1_191',
 'GEN_BP_Box2_1_192',
 'GEN_BP_Box2_1_311',
 'GEN_BP_Box2_1_312',
 'GEN_BP_Box2_2_134',
 'GEN_BP_Box2_2_147',
 'GEN_BP_Box2_2_166',
 'GEN_BP_Box2_3_165',
 'GEN_BP_Box2_4_100',
 'GEN_BP_Box3_1_281',
 'GEN_BP_Box3_1_348',
 'GEN_BP_Box3_1_90',
 'GEN_BP_Box3_3_111',
 'GEN_BP_Box3_3_112',
 'GEN_BP_Box3_3_223',
 'GEN_BP_Box3_3_247',
 'GEN_BP_Box3_3_248',
 'GEN_BP_Box3_3_282',
 'GEN_BP_Box3_3_347',
 'GEN_BP_Box3_3_81',
 'GEN_BP_Box3_4_143',
 'GEN_BP_Box3_4_224',
 'GEN_BP_Box3_4_279',
 'GEN_BP_Box3_4_280',
 'GEN_BP_Box3_4_96',
 'GEN_BP_Box_1_87',
 'GEN_BP_Box_4_154',
 'GEN_BP_Building_05_C_26',
 'GEN_BP_Building_09_C_49',
 'GEN_BP_Building_101_C_51',
 'GEN_BP_Building_102_C_32',
 'GEN_BP_Building_103_C_70',
 'GEN_BP_Building_105_C_46',
 'GEN_BP_Building_107_C_44',
 'GEN_BP_Building_108_C_63',
 'GEN_BP_Building_10_C_23',
 'GEN_BP_Building_116_C_11',
 'GEN_BP_Building_11_C_48',
 'GEN_BP_Building_11_C_65',
 'GEN_BP_Building_122_C_38',
 'GEN_BP_Building_126_C_12',
 'GEN_BP_Bu

## Traffic Simulation

In [5]:
traffic_controller = TrafficController(config, 15, 15, "/data4/dylanlu/developer/SimWorld/simworld/data/roads.json", 1, 0.1)
traffic_controller.init_communicator(communicator)

2025-12-05 18:38:09,126 - SimWorld.TrafficController - INFO - TrafficController initialized with 15 vehicles and 15 pedestrians
2025-12-05 18:38:09,129 - SimWorld.TrafficController - INFO - Loading road network from file: /data4/dylanlu/developer/SimWorld/simworld/data/roads.json
2025-12-05 18:38:09,131 - SimWorld.TrafficController - INFO - Found 10 roads in map file
2025-12-05 18:38:09,133 - SimWorld.TrafficController - INFO - Road 0: Start=(0.0, 0.0), End=(20000.0, 0.0)
2025-12-05 18:38:09,134 - SimWorld.TrafficController - INFO - Road 1: Start=(0.0, 0.0), End=(-20000.0, 0.0)
2025-12-05 18:38:09,135 - SimWorld.TrafficController - INFO - Road 2: Start=(20000.0, 0.0), End=(40000.0, 0.0)
2025-12-05 18:38:09,137 - SimWorld.TrafficController - INFO - Road 3: Start=(20000.0, 0.0), End=(20000.0, -20000.0)
2025-12-05 18:38:09,138 - SimWorld.TrafficController - INFO - Road 4: Start=(-20000.0, 0.0), End=(-20000.0, -20000.0)
2025-12-05 18:38:09,139 - SimWorld.TrafficController - INFO - Road 5: 

In [6]:
map = Map(config, traffic_signals=traffic_controller.traffic_signals)
humanoid = Humanoid(position=Vector(1700, 1700), direction=Vector(1, 0), map=map, communicator=communicator, config=config)

In [7]:
communicator.spawn_agent(humanoid, config['user.model_path'])

(1700.0, 1700.0, 600)


In [8]:
traffic_controller.spawn_objects_in_unreal_engine()
communicator.spawn_ue_manager(config['simworld.ue_manager_path'])

2025-12-05 18:38:35,178 - SimWorld.TrafficController - INFO - Vehicles spawned
2025-12-05 18:38:49,596 - SimWorld.TrafficController - INFO - Pedestrians spawned
2025-12-05 18:39:05,477 - SimWorld.TrafficController - INFO - Traffic signals spawned


In [9]:
def update_states():
        vehicle_ids = [vehicle.id for vehicle in traffic_controller.vehicles]
        pedestrian_ids = [pedestrian.id for pedestrian in traffic_controller.pedestrians]
        traffic_signal_ids = [signal.id for signal in traffic_controller.traffic_signals]
        humanoid_ids = [humanoid.id]
        result = traffic_controller.communicator.get_position_and_direction(vehicle_ids, pedestrian_ids, traffic_signal_ids, humanoid_ids)
        for (type, object_id), values in result.items():
            if type == 'vehicle':
                position, direction = values
                traffic_controller.vehicles[object_id].position = position
                traffic_controller.vehicles[object_id].direction = direction
            elif type == 'pedestrian':
                position, direction = values
                traffic_controller.pedestrians[object_id].position = position
                traffic_controller.pedestrians[object_id].direction = direction
            elif type == 'traffic_signal':
                is_vehicle_green, is_pedestrian_walk, left_time = values
                for signal in traffic_controller.traffic_signals:
                    if signal.id == object_id:
                        if is_vehicle_green:
                            signal.set_state((TrafficSignalState.VEHICLE_GREEN, TrafficSignalState.PEDESTRIAN_RED))
                        elif is_pedestrian_walk:
                            signal.set_state((TrafficSignalState.VEHICLE_RED, TrafficSignalState.PEDESTRIAN_GREEN))
                        else:
                            signal.set_state((TrafficSignalState.VEHICLE_RED, TrafficSignalState.PEDESTRIAN_RED))
                        signal.set_left_time(left_time)
                        break
            elif type == 'humanoid':
                pos, dir = values
                humanoid.position = pos
                humanoid.direction = dir

In [10]:
exit_event = threading.Event()

In [11]:
t = threading.Thread(target=traffic_controller.simulation, args=(update_states, exit_event))
t.start()

2025-12-05 18:39:26,198 - SimWorld.TrafficController - INFO - Starting simulation


## Action Planner

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = '<your_openai_api_key>'

In [16]:
llm = A2ALLM(model_name='gpt-4o', provider='openai')
map.initialize_map_from_file(roads_file='/data4/dylanlu/developer/SimWorld/simworld/data/roads.json')  # use default map

2025-12-05 18:40:23,540 - SimWorld.BaseLLM - INFO - Initialized LLM client for model -- gpt-4o, url -- default


In [17]:
action_planner = LocalPlanner(agent=humanoid, model=llm, rule_based=False)

In [18]:
communicator.spawn_object('GEN_BP_Box_1_C', '/Game/InteractableAsset/Box/BP_Interactable_Box.BP_Interactable_Box_C', (1700, -1700, 20), (0, 0, 0))

In [19]:
plan = 'Go to (1700, -1700) and pick up GEN_BP_Box_1_C.'
action = action_planner.parse(plan)

2025-12-05 18:40:36,660 - SimWorld.LocalPlanner - INFO - Agent 0 Response: {"action_queue":[1,2],"destination":{"x":1700.0,"y":-1700.0},"object_name":"GEN_BP_Box_1_C","reasoning":"I need to navigate to the destination (1700, -1700) and pick up the object GEN_BP_Box_1_C."}, call time: 1.136528730392456
2025-12-05 18:40:36,661 - SimWorld.LocalPlanner - INFO - Agent 0 Actions: HighLevelActionSpace(destination=Vector(x=1700.0, y=-1700.0), object_name=GEN_BP_Box_1_C, action_queue=[1, 2], reasoning=I need to navigate to the destination (1700, -1700) and pick up the object GEN_BP_Box_1_C.)


In [20]:
### Test executor
action_planner.execute(action)

2025-12-05 18:40:57,979 - SimWorld.LocalPlanner - INFO - Agent 0 Target destination: Vector(x=1700.0, y=-1700.0)
2025-12-05 18:40:57,981 - SimWorld.LocalPlanner - INFO - Agent 0 is navigating to Vector(x=1700.0, y=-1700.0), current position: Vector(x=1700.0, y=1700.0), vision based mode
2025-12-05 18:41:07,069 - SimWorld.LocalPlanner - INFO - Agent 0 is taking action LowLevelActionSpace(choice=TURN_AROUND, duration=None, direction=None, angle=90.0, clockwise=False, reasoning=I need to turn left to face the direction of the destination, as the relative angle is -90 degrees. This will align me properly to continue moving along the sidewalk towards the destination.)
2025-12-05 18:41:14,816 - SimWorld.LocalPlanner - INFO - Agent 0 is taking action LowLevelActionSpace(choice=TURN_AROUND, duration=None, direction=None, angle=90.0, clockwise=True, reasoning=I see a crosswalk ahead. I should turn right to align with the crosswalk and move safely across the intersection.)
2025-12-05 18:41:23,79

KeyError: 'VehicleCollision'

In [21]:
exit_event.set()
t.join()

2025-12-05 18:46:33,392 - SimWorld.TrafficController - INFO - Stopping simulation
2025-12-05 18:46:39,331 - SimWorld.TrafficController - INFO - Simulation ended


In [22]:
communicator.disconnect()